In [1]:
import pandas as pd

df = pd.read_csv("books_with_categories.csv")

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)
classifier("I love this!")


Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119900392368436},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [14]:
preds = classifier(df["description"][0].split("."))

In [17]:
preds

[[{'label': 'surprise', 'score': 0.7296020984649658},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816228479146957},
  {'label': 'joy', 'score': 0.0479426383972168},
  {'label': 'anger', 'score': 0.009156371466815472},
  {'label': 'disgust', 'score': 0.0026284765917807817},
  {'label': 'sadness', 'score': 0.002122163539752364}],
 [{'label': 'neutral', 'score': 0.4493706524372101},
  {'label': 'disgust', 'score': 0.2735912799835205},
  {'label': 'joy', 'score': 0.10908322036266327},
  {'label': 'sadness', 'score': 0.09362740069627762},
  {'label': 'anger', 'score': 0.04047828167676926},
  {'label': 'surprise', 'score': 0.026970166712999344},
  {'label': 'fear', 'score': 0.006879047024995089}],
 [{'label': 'neutral', 'score': 0.6462154984474182},
  {'label': 'sadness', 'score': 0.24273382127285004},
  {'label': 'disgust', 'score': 0.04342272877693176},
  {'label': 'surprise', 'score': 0.028300544247031212},
  {'label': 'joy', 'score': 0.014211482

In [32]:
from tqdm import tqdm

isbns = []

def get_max_emotion_score(emotion, sentence_pred):
    scores = []
    for i in sentence_pred:
        for j in i:
            if j["label"] == emotion:
                scores.append(j["score"])
    return max(scores)


emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]


emotions_dict = {emotion: [] for emotion in emotions}


for i in tqdm(range(len(df))):
    isbns.append(df["isbn13"][i])
    sentences = df["description"][i].split(".")
    sentence_pred = classifier(sentences, batch_size=8)  # batching
    for emotion in emotions_dict:
        max_score = get_max_emotion_score(emotion, sentence_pred)
        emotions_dict[emotion].append(max_score)



  1%|          | 55/6397 [00:16<31:32,  3.35it/s] 


KeyboardInterrupt: 

In [30]:
emotions_dict

{'anger': array([0.04047828, 0.61261988, 0.01603621, 0.35148466, 0.0814124 ,
        0.53818434, 0.13283803, 0.        ]),
 'disgust': array([0.27359128, 0.34828481, 0.0606952 , 0.15072225, 0.18449552,
        0.72717494, 0.064666  , 0.        ]),
 'fear': array([0.9281683 , 0.9723208 , 0.00191786, 0.36070606, 0.04019525,
        0.26585764, 0.74742717, 0.        ]),
 'joy': array([0.93279803, 0.7672382 , 0.2518813 , 0.02480991, 0.035207  ,
        0.87256557, 0.00796585, 0.        ]),
 'sadness': array([0.96715754, 0.06179974, 0.02098823, 0.47588021, 0.16030179,
        0.1565351 , 0.40800145, 0.        ]),
 'surprise': array([0.7296021 , 0.25254625, 0.02968322, 0.07487808, 0.07487808,
        0.27190357, 0.02882093, 0.        ]),
 'neutral': array([0.6462155 , 0.88793951, 0.73268509, 0.56766838, 0.8843897 ,
        0.71219414, 0.38535854, 0.        ])}

In [31]:
isbns

[9780002005883,
 9780002261982,
 9780006163831,
 9780006178736,
 9780006280897,
 9780006280934,
 9780006380832,
 9780006470229]